# Chapter 12: Routing and Orchestration

**Scenario:** You are building a **Super Assistant** for a luxury hotel. Instead of having one giant, confusing prompt that tries to handle everything, you want to build a system that **routes** the user to the right "Expert" (Billing, Concierge, or Tech Support).

In this notebook, you will learn:
- 🚦 **Routing**: How to make the AI choose the best path for a request.
- 🏘️ **Specialized Experts**: Building small, highly effective prompts for specific tasks.
- 🔗 **Orchestration**: Combining multiple steps into a seamless experience.

---

## 1. Setup

Standard setup.

In [1]:
# Install tools (if needed)
%pip install -q litellm python-dotenv

/Users/param/learn/learnwithparam/lwp-workshops/practical-prompt-engineering-workshop/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# Suppress noisy debug logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

load_dotenv()
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')
print(f"Ready to route requests with: {MODEL_NAME}")

Ready to route requests with: openrouter/google/gemini-2.0-flash-001


In [3]:
def get_completion(prompt, system_prompt=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

## 2. The Router Prompt

Large prompts become "messy" and start ignoring instructions. Routing solves this by spliting the problem. 

Step 1 is always identifying the **Intent**.

In [4]:
def route_request(query):
    prompt = f"""
    Classify the following hotel guest query into one of these categories:
    - BILLING: Invoices, extra charges, payment methods.
    - CONCIERGE: Restaurant bookings, tours, spa, baggage.
    - TECH: Wifi issues, TV remote, smart room controls.
    
    Query: {query}
    
    Output ONLY the category name.
    """
    category = get_completion(prompt).strip().upper()
    return category

query = "The Wifi in my room isn't connecting to my laptop."
intent = route_request(query)
print(f"Detected Intent: {intent}")

Detected Intent: TECH


## 3. Specialized Experts

Now that we know the intent, we can send the query to a prompt designed *only* for that category. This makes the response much more accurate and professional.

In [5]:
EXPERTS = {
    "BILLING": "You are a Hotel Accountant. You are precise, professional, and explain charges clearly.",
    "CONCIERGE": "You are a Luxury Hotel Concierge. You are helpful, enthusiastic, and local expert.",
    "TECH": "You are a Hotel IT Specialist. You provide clear, step-by-step instructions for devices."
}

def handle_query(query):
    # 1. Route
    intent = route_request(query)
    
    # 2. Select Persona
    system_prompt = EXPERTS.get(intent, "You are a helpful hotel assistant.")
    
    # 3. Generate Answer
    response = get_completion(query, system_prompt=system_prompt)
    return intent, response

test_queries = [
    "Why was I charged $50 for the minibar?",
    "Can you suggest a good Italian place nearby?",
    "My TV is stuck on a blue screen."
 ]

for q in test_queries:
    category, answer = handle_query(q)
    print(f"\n[{category}] Query: {q}")
    print(f"Answer: {answer}")


[BILLING] Query: Why was I charged $50 for the minibar?
Answer: Good morning, thank you for contacting me.

I'd be happy to look into that minibar charge for you. According to our records, the minibar in your room was accessed during your stay. Specifically, it appears that one bottle of sparkling water and one snack pack were consumed. The sparkling water is priced at $20 and the snack pack is $30, totaling $50.

Please let me know if you have any further questions.


[CONCIERGE] Query: Can you suggest a good Italian place nearby?
Answer: Ah, a craving for Italian! Excellent choice. To give you the best recommendation, I need a little more information. Are you looking for:

*   **A specific vibe?** (Romantic, family-friendly, lively, casual)
*   **A particular type of Italian cuisine?** (Classic red sauce, regional specialties, modern interpretations)
*   **A price range?** (Budget-friendly, mid-range, splurge-worthy)

In the meantime, here are a couple of popular options nearby that

## 4. Orchestration (The Multi-Step Flow)

Orchestration usually means managing a sequence of events. For a luxury hotel, we might want to:
1. Categorize the request.
2. Determine the **Urgency**.
3. Format a task for the staff.


In [6]:
def orchestrate_task(query):
    # Step 1: Analyze Urgency
    urgency_prompt = f"Rate the urgency of this hotel request from 1 (Casual) to 5 (Critical). Output only the number. Query: {query}"
    priority = get_completion(urgency_prompt).strip()
    
    # Step 2: Categorize
    category = route_request(query)
    
    # Step 3: Format staff notification
    staff_prompt = f"""
    Format a task for the staff.
    Task: {query}
    Category: {category}
    Priority: {priority}
    Response Tone Required: {'Immediate Action' if priority >= '4' else 'Standard'}
    """
    
    return get_completion(staff_prompt)

alert = orchestrate_task("There is water leaking from my ceiling!")
print(alert)

## TASK: WATER LEAK - IMMEDIATE ACTION REQUIRED

**Category:** CONCIERGE

**Priority:** 5 (CRITICAL - IMMEDIATE ACTION REQUIRED)

**Description:**

A guest is reporting a water leak from their ceiling. This is a critical issue that requires immediate attention to prevent further damage and ensure guest safety and comfort.

**Instructions:**

1.  **Immediately dispatch a member of the maintenance team to the guest's room.** (Note: If maintenance is unavailable, the concierge should personally assess the situation and take initial steps as outlined below).
2.  **Identify the source of the leak.**
3.  **Take immediate steps to contain the leak and minimize damage.** This may include:
    *   Placing buckets or containers to catch the water.
    *   Covering furniture and belongings with protective sheeting.
    *   Shutting off the water supply to the affected area (if safe and possible).
4.  **Communicate clearly and empathetically with the guest.** Reassure them that the issue is being 

## 5. Exercise: The Smart Router

**Goal:** Build a router for a **Support Ticket System**.

**Requirement:**
- Categorize the user message into `BUG`, `FEATURE_REQUEST`, or `QUESTION`.
- If it's a `BUG`, output a formatted bug report (summary, steps).
- If it's a `QUESTION`, answer it politely.

**Input:** "The login button doesn't work when I use my phone, it just refreshes the page."

In [7]:
user_msg = "The login button doesn't work when I use my phone, it just refreshes the page."

# 1. Write your routing prompt
router_prompt = "..."

# 2. Write your logic

# 3. Test it
print("Testing...")

Testing...


## Summary

1.  **Don't build "Mega-Prompts"**: Large prompts are unreliable. Route to specialized experts instead.
2.  **Routing is Step 1**: Identifying the intent allows you to pick the best tools and instructions.
3.  **Orchestrate Workflows**: Combine multiple calls to add logic (like Priority or Formatting) to your AI system.